##### duration: ~1/4 to 1/2 real time

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/kpms/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run

In [6]:
for idx, recording_row in recording_df.iterrows():
    break

In [7]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [8]:
output_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline')

In [9]:
output_directory_sizenorm = output_directory / 'size_normalization' / recording_row.video_recording_id
output_directory_sizenorm.mkdir(parents=True, exist_ok=True)

In [10]:
from multicamera_airflow_pipeline.tim_240731.keypoints.size_norm import SizeNormalizer 

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.size_norm:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/kpms/bin/python3


In [11]:
!ls {output_directory}

2D_predictions	    continuous_features   size_normalization  triangulation
arena_alignment     egocentric_alignment  spikesorting	      triangulation2
camera_calibration  gimbal		  tensorrt
camera_sync	    openephys_sync	  tensorrt-2


In [12]:
!ls { output_directory / 'gimbal' / recording_row.video_recording_id}

completed.log			gimbal_success.bool.10000x1.mmap
fit_likelihood.jpg		gimbal_success.bool.90009x1.mmap
gimbal.float32.10000x25x3.mmap	joint_directions.jpg
gimbal.float32.90009x25x3.mmap	keypoints_order_gimbal.npy
gimbal_params.p			skeleton_distances.jpg


In [13]:
predictions_3d_file = list(
    (output_directory / "gimbal" / recording_row.video_recording_id).glob(
        "gimbal.*.mmap"
    )
)[0]

In [14]:
size_normalizer = SizeNormalizer(
    predictions_3d_file = predictions_3d_file,
    size_norm_output_directory = output_directory_sizenorm,
)

In [15]:
size_normalizer.run()

calculating joint lengths:   0%|          | 0/26 [00:00<?, ?it/s]

calculating angles:   0%|          | 0/10000 [00:00<?, ?it/s]

computing new keypoint positions:   0%|          | 0/26 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

...

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

adding bone length:   0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
!ls {output_directory_sizenorm}

size_norm_angles.float32.10000x25x3.mmap  size_norm.float32.10000x25x3.mmap
size_norm_angles.float32.90009x25x3.mmap  size_norm.float32.90009x25x3.mmap


In [37]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt